In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:

train=pd.read_csv('/home/aistudio/data/data18783/train.csv')

In [ ]:
train.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [ ]:
test=pd.read_csv('/home/aistudio/data/data18783/test.csv')


In [ ]:
test.columns

Index(['id', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [ ]:
train = pd.concat([train, test], ignore_index=True, sort=False)

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
train=reduce_mem_usage(train)

Mem. usage decreased to 3971.28 Mb (46.4% reduction)


In [5]:

train['time']=pd.to_datetime(train['hour'],format='%y%m%d%H')
train['day']=train['time'].dt.day
train['hour']=train['time'].dt.hour


#train['user']=train['device_id'].astype(str)+ '_' + train['device_ip'].astype(str)+ '_' +train['device_model'].astype(str)
#train['app']=train['app_id'].astype(str)+ '_' + train['app_domain'].astype(str)+ '_' +train['app_category'].astype(str)
#train['site']=train['site_id'].astype(str)+ '_' + train['site_domain'].astype(str)+ '_' +train['site_category'].astype(str)




In [ ]:
#测试时用,只抽两天做训练集
matrix=train[(train.day==31)|(train.day==30)|(train.day==29)]
del train,test
gc.collect()

173

In [ ]:
#测试时用


tra=matrix[(matrix.day==30)|(matrix.day==29)]
test=matrix[(matrix.day==31)]
tra.drop(['id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)
test.drop(['day'], axis=1, inplace=True)
tra.drop(['day'], axis=1, inplace=True)
del matrix
gc.collect()

66

In [ ]:
#转化为libffm格式
from collections import defaultdict
class FFMFormatPandas:
    def __init__(self):
        self.field_index_ = None
        self.feature_index_ = None
        self.y = None

    def fit(self, df, y=None):
        self.y = y
        df_ffm = df[df.columns.difference([self.y])]
        
        if self.field_index_ is None:
            self.field_index_ =  defaultdict(int)
            for i, col in enumerate(df_ffm):
                self.field_index_[col] =i
            

        if self.feature_index_ is not None:
            last_idx = max(list(self.feature_index_.values()))

        if self.feature_index_ is None:
            self.feature_index_ = defaultdict(int)
            last_idx = 0

        for col in df.columns:
            vals = df[col].unique()
            for val in vals:
                if pd.isnull(val):
                    continue
                name = '{}_{}'.format(col, val)
                if name not in self.feature_index_:
                    self.feature_index_[name] = last_idx
                    last_idx += 1
            self.feature_index_[col] = last_idx
            last_idx += 1
        return self

    def fit_transform(self, df, y=None):
        self.fit(df, y)
        return self.transform(df)

    def transform_row_(self, row, t):
        ffm = []
        if self.y != None:
            ffm.append(str(row.loc[row.index == self.y][0]))
        if self.y is None:
            ffm.append(str(0))

        for col, val in row.loc[row.index != self.y].to_dict().items():
            col_type = t[col]
            name = '{}_{}'.format(col, val)
            if col_type.kind ==  'O':
                ffm.append('{}:{}:1'.format(self.field_index_[col], self.feature_index_[name]))
            elif col_type.kind == 'i':
                ffm.append('{}:{}:{}'.format(self.field_index_[col], self.feature_index_[col], val))
        return ' '.join(ffm)

    def transform(self, df):
        t = df.dtypes.to_dict()
        return pd.Series({idx: self.transform_row_(row, t) for idx, row in df.iterrows()})



In [ ]:
#ffm格式所需
test.loc[:,]['click']=0
test['click'].value_counts()

0    4577464
Name: click, dtype: int64

In [ ]:
test.head()

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,time
40428967,0,0,1005,0,235ba823,f6ebf28e,f028772b,ecad2386,7801e8d9,07d7df22,...,0,8330,320,50,761,3,175,100075,23,2014-10-31
40428968,0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,0,22676,320,50,2616,0,35,100083,51,2014-10-31
40428969,0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,0,22676,320,50,2616,0,35,100083,51,2014-10-31
40428970,0,0,1005,0,85f751fd,c4e18dd6,50e219e0,51cedd4e,aefc06bd,0f2161f8,...,0,18648,320,50,1092,3,809,100156,61,2014-10-31
40428971,0,0,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,f95efa07,...,0,23160,320,50,2667,0,47,-1,221,2014-10-31


In [ ]:
#转化为ffm所需的格式
for feat in ['hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'time']:

    tra.loc[:,feat]=tra.loc[:][feat].map(str)
    test.loc[:,feat]=test.loc[:][feat].map(str)
   


ffm_train = FFMFormatPandas()
ffm_train_data = ffm_train.fit_transform(tra, y='click')
ffm_test_data= ffm_train.transform(test)


ffm_train_data.to_csv("tralibffm.csv", index=False)
ffm_test_data.to_csv("testlibffm.csv",index=False)




In [ ]:
ffm_train_data.to_csv("tra2libffm.csv", index=False)

In [ ]:
#这里使用libffm工具进行计算，这里略过

In [ ]:
#生成submission
submission=pd.read_csv('/home/aistudio/data/data18783/Submission.csv')
output=pd.read_csv('/home/aistudio/data/data18783/output',header=None)
submission['click']=output
submission.to_csv("/home/aistudio/data/data18783/submission.csv", index=False)